In [5]:
import cv2
import mediapipe as mp
import math

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Eye landmarks indexes (MediaPipe Face Mesh)
LEFT_EYE = [362, 385, 387, 263, 373, 380]
RIGHT_EYE = [33, 160, 158, 133, 153, 144]

# Thresholds
EAR_THRESHOLD = 0.25
CONSEC_FRAMES_LIMIT = 40

# Counter for frames with eyes closed
closed_eyes_frames = 0

def euclidean_distance(point1, point2):
    return math.sqrt((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2)

def calculate_ear(landmarks, points, width, height):
    coords = [(int(landmarks.landmark[i].x * width), int(landmarks.landmark[i].y * height)) for i in points]
    vertical_1 = euclidean_distance(coords[1], coords[5])
    vertical_2 = euclidean_distance(coords[2], coords[4])
    horizontal = euclidean_distance(coords[0], coords[3])
    ear = (vertical_1 + vertical_2) / (2.0 * horizontal)
    return ear

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    height, width = frame.shape[:2]
    
    # Process frame with MediaPipe
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_ear = calculate_ear(face_landmarks, LEFT_EYE, width, height)
            right_ear = calculate_ear(face_landmarks, RIGHT_EYE, width, height)
            avg_ear = (left_ear + right_ear) / 2.0
            
            # Show EAR on screen
            cv2.putText(frame, f'EAR: {avg_ear:.2f}', (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)
            
            # Check if eyes are closed
            if avg_ear < EAR_THRESHOLD:
                closed_eyes_frames += 1
                if closed_eyes_frames > CONSEC_FRAMES_LIMIT:
                    cv2.putText(frame, "ALERT! Wake Up!", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,0,255), 3)
            else:
                closed_eyes_frames = 0
    else:
        closed_eyes_frames = 0
    
    cv2.imshow("Drowsiness Alert", frame)
    
    if cv2.waitKey(1) & 0xFF == 27:  # Press ESC to exit
        break

cap.release()
cv2.destroyAllWindows()
